In [63]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import os,warnings,sys,itertools,datetime,time,timeit

import torch
from torch import nn,optim
import torch.functional as F

import re
import unicodedata

import pandas as pd

import _pickle as pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
root_path = '../../../TF_Data/Dropbox/TF_data/SEQ2SEQ/de-en/'
#root_path = '../../../../../../../TF_data/SEQ2SEQ/de-en/''

In [30]:
with open(root_path+'europarl-v7.de-en.en','r',encoding='utf-8') as file:
    data_en = file.read()
with open(root_path+'europarl-v7.de-en.de','r',encoding='utf-8') as file:
    data_de = file.read()

In [57]:
class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.index2word = {0:'SOS',1:'EOS'}
        self.word2count = {}
        self.nwords = 2
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.nwords
            self.index2word[self.nwords] = word
            self.word2count[word] = 1
            self.nwords += 1
        else:
            self.word2count[word] += 1
    def addSentence(self,sent):
        for word in sent:
            self.addWord(word)

In [58]:
def normalizeString(s):
    s = s.lower().strip()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r'([.!?])',r' \1',s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ',s)
    return s

In [68]:
LOAD_PREPARED_LANGUAGE_MODEL = True

if not LOAD_PREPARED_LANGUAGE_MODEL:
    df = pd.DataFrame(data_en.split('\n'),data_de.split('\n')).reset_index(drop=False).rename(columns={
        'index':'DE',
        0:'EN'
    })
    
    input_lang = Lang('de')
    output_lang = Lang('en')
    
    for r,cs in df.iterrows():
        input_lang.addSentence(normalizeString(cs.DE))
        output_lang.addSentence(normalizeString(cs.EN))
    
    pickle.dump(input_lang,open('./data/input_lang.pkl','wb'))
    pickle.dump(output_lang,open('./data/output_lang.pkl','wb'))
    df.to_pickle('./data/pairs.pkl')
else:
    input_lang = pickle.load(open('./data/input_lang.pkl','rb'))
    output_lang = pickle.load(open('./data/output_lang.pkl','rb'))
    df = pickle.load(open('./data/pairs.pkl','rb'))

EOFError: Ran out of input

In [65]:
pickle.dump(input_lang,op